# Transferring Topo stuff to python

In [ ]:
# Import packages 
import os
import sys
workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature




from myPythonTools.TopoAna import Plots as Po

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.tri as tri
import matplotlib.colors as colors

from scipy.io import FortranFile

import importlib
import glob
import copy

importlib.reload(Po)

#sys.path.append('../Plotting/')
#   Now you can imprt modules in ../Plotting
#import xyp_plot as xyp
#import ana as a


In [ ]:
host=os.getenv('HOST')
host

## Set some paths and directories

In [ ]:

"""
TopoRoot = '/glade/work/juliob/Topo/MyTopo/'
Case = 'ne120pg3_Sco25_Test03'
ogrid= 'ne120pg3'
Case = 'ne30pg3_Sco100_Test01'
ogrid= 'ne30pg3'
#Case = 'ne240pg3_Sco14_Test02'
#ogrid= 'ne240pg3'
"""

TopoRoot = '/glade/work/juliob/Topo/NCARTopo_jtbDevel/'
Case = 'ne120pg3_Sco25_X02'
ogrid= 'ne120pg3'
#Case = 'ne30pg3_Sco100_Test01'
#ogrid= 'ne30pg3'


In [ ]:
foo=glob.glob( TopoRoot +'/cases/'+ Case + '/output/'+ogrid+'*.nc')
FileN = foo[-1]
print(FileN)
dCAM=xr.open_dataset( FileN )
print( list(dCAM.variables) )
print(np.shape(dCAM['lat']))
lonc=dCAM['lon'].values
latc=dCAM['lat'].values
angl=dCAM['ANGLL'].values
hwdt=dCAM['HWDTH'].values
clng=dCAM['CLNGT'].values

CAM_Topo_FileN=FileN


In [ ]:
print( foo )

In [ ]:
Po.CAMridgelet(lat=60.,lon=0.,angll=45.)
plt.plot(dCAM.ISOVAR.values.flatten())

In [ ]:
fig,ax=plt.subplots( 2,2, figsize=(16,12) )

ax=ax.flatten()
dlevs = np.linspace( 0,2000, num=21 )
wlevs = np.linspace( 0,1, num=11 )

cmap='inferno'

c0=ax[0].tricontourf( lonc, latc, dCAM.ISOVAR.values ,levels=dlevs, cmap=cmap )
c1=ax[1].tricontourf( lonc, latc, dCAM.ISOWGT.values ,levels=wlevs, cmap=cmap )
c2=ax[2].tricontourf( lonc, latc, dCAM.SGH.values ,levels=dlevs, cmap=cmap  )
c3=ax[3].tricontourf( lonc, latc, dCAM.MXDIS[0,:].values ,levels=dlevs, cmap=cmap  )

plt.colorbar(c0 )
plt.colorbar(c1 )
plt.colorbar(c2 )
plt.colorbar(c3 )



In [ ]:
foo=glob.glob( TopoRoot +'/cases/'+ Case + '/output/topo*.nc')
FileN = foo[-1]
print(FileN)
topo=xr.open_dataset( FileN )
print(list(topo.variables))
print(topo.attrs)
raw=topo['terr_dev']+topo['terr_sm']
Loo = np.size(raw)
nc=int(np.sqrt( Loo/6 ))
print(int(nc))
raw=np.reshape( raw.values , (6,nc,nc) )
dev=np.reshape( topo['terr_dev'].values , (6,nc,nc) )
smoo=np.reshape( topo['terr_sm'].values , (6,nc,nc) )
tlats=np.reshape( topo['lat'].values , (6,nc,nc) )
tlons=np.reshape( topo['lon'].values , (6,nc,nc) )

# Define the range of indices
x_indices = np.arange(3000)
y_indices = np.arange(3000)

# Use meshgrid to create 2D arrays of x and y indices
xgrid, ygrid = np.meshgrid(x_indices, y_indices, indexing='xy')


In [ ]:
# Ste up locations for map plots


#plt.axes( projection=ccrs.PlateCarree(central_longitude=0))
ValidLocation=True
#location='SouthwestUS'
#location='SouthAndesAntarcticPen'
#location='McMurdo'
#location='Antarctica'
location='Colorado'
if (location==None):
    print( "No valid location" )
    ValidLocation=False
elif (location=='Colorado'):
    panel=4
    lonrange=np.asarray([-117.,-100] )+360. #-2
    latrange=np.asarray([30,41]) #-4 #-4
    central_longitude = np.average(lonrange)
    central_latitude = np.average(latrange)
    MapProj = ccrs.LambertConformal(central_longitude=central_longitude,central_latitude=central_latitude)
elif (location=='SouthwestUS'):
    panel=4
    lonrange=np.asarray([-130.,-104] )+360. #-2
    latrange=np.asarray([29,42]) #-4 #-4
    central_longitude = np.average(lonrange)
    central_latitude = np.average(latrange)
    MapProj = ccrs.AlbersEqualArea(central_longitude=central_longitude,central_latitude=central_latitude)
elif (location=='SouthAndesAntarcticPen'):
    panel=5
    lonrange=np.asarray( [260,300] ) #-2
    latrange=np.asarray([-75,-50]) #-4 #-4
    central_longitude = np.average(lonrange)
    central_latitude = np.average(latrange)
    MapProj = ccrs.Orthographic(central_longitude=central_longitude,central_latitude=central_latitude)
elif (location=='McMurdo'):
    panel=5
    lonrange=np.asarray( [120,230] ) #-2
    latrange=np.asarray([-87,-50]) #-4 #-4
    central_longitude = np.average(lonrange)
    central_latitude = np.average(latrange)
    MapProj = ccrs.Orthographic(central_longitude=central_longitude,central_latitude=central_latitude)
elif (location=='Antarctica'):
    panel=5
    lonrange=np.asarray( [0,360] ) #-2
    latrange=np.asarray([-90,-50]) #-4 #-4
    central_longitude = 180. # np.average(lonrange)
    central_latitude = -90. #np.average(latrange)
    MapProj = ccrs.Orthographic(central_longitude=central_longitude,central_latitude=central_latitude)
else:
    print( "No valid location" )
    ValidLocation=False

"""
panel=5
#lonrange=np.asarray( [110,250] ) #-2
#latrange=np.asarray([-85,-60]) #-4 #-4
lonrange=np.asarray( [220,340] ) #-2
latrange=np.asarray([-85,-50]) #-4 #-4
"""


if ( ValidLocation==True ):
    ip=panel-1
    """
    lonrange=np.asarray([0,40]) #+360 #-2
    latrange=np.asarray([-40,-0]) #-4
    ip=0
    """
    zoo=np.where( ( (lonc<=lonrange[1])&(lonc>lonrange[0]) )&( (latc<=latrange[1])&(latc>latrange[0]) ) )
    
    yzoo = np.where(  (tlons[ip,:,:]<=lonrange[1])&(tlons[ip,:,:]>lonrange[0])&(tlats[ip,:,:]<=latrange[1])&(tlats[ip,:,:]>latrange[0]) , ygrid , -1)
    xzoo = np.where(  (tlons[ip,:,:]<=lonrange[1])&(tlons[ip,:,:]>lonrange[0])&(tlats[ip,:,:]<=latrange[1])&(tlats[ip,:,:]>latrange[0]) , xgrid , -1)
    print(len(zoo[0]))
    iroo=zoo[0]
    print(np.shape(angl))
    
    xxrange = [ np.min( xzoo[xzoo>=0] ) , np.max( xzoo[xzoo>=0] ) ]
    print(xxrange)
    yyrange = [ np.min( yzoo[yzoo>=0] ) , np.max( yzoo[yzoo>=0] ) ]
    print(yyrange)

print(central_latitude)

In [ ]:
#MapProj = MapProj = ccrs.PlateCarree(central_longitude=180.)
#MapProj = MapProj = ccrs.Orthographic(central_longitude=180.,central_latitude=-70.)
#MapProj = ccrs.LambertConformal(central_longitude=central_longitude,central_latitude=central_latitude)
#MapProj = ccrs.AlbersEqualArea(central_longitude=central_longitude,central_latitude=central_latitude)
#MapProj = ccrs.Orthographic(central_longitude=central_longitude,central_latitude=central_latitude)
#MapProj = ccrs.PlateCarree(central_longitude=central_longitude)

DataProj = ccrs.PlateCarree()
#plt.figure( figsize=(21,21))
fig = plt.figure(figsize=( 15,15 ))

#xx=np.asarray([1000,2300] )
#yy=np.asarray([500,1500] )

xx=xxrange # np.asarray([000,1000] )
yy=yyrange #np.asarray([1200,2200] )

Axes=[.1,.1,.8,.8]
ax = fig.add_axes( Axes , projection=MapProj)

#plt.contour( tlons[ip,:,:],tlats[ip,:,:],dev[ip,:,:], levels=np.linspace(-100,500.,num=21), cmap='terrain' ,alpha=.5) #,
ax.contourf( tlons[ip , yy[0]:yy[1],xx[0]:xx[1] ],tlats[ip , yy[0]:yy[1],xx[0]:xx[1] ],raw[ip, yy[0]:yy[1],xx[0]:xx[1] ], levels=np.linspace(-100,4000.,num=21), cmap='terrain',alpha=.5 , transform=DataProj ) #,
#ax.contourf( tlons[ip , yy[0]:yy[1],xx[0]:xx[1] ],tlats[ip , yy[0]:yy[1],xx[0]:xx[1] ],dev[ip, yy[0]:yy[1],xx[0]:xx[1] ], levels=np.linspace(-500,500.,num=21), cmap='terrain',alpha=.5 , transform=DataProj ) #,
#plt.xlim(lonrange)
#plt.ylim(latrange)
#plt.plot( [40,90],[0,30])
for ir in iroo:
    for rank in np.arange(10):
        lat0,lon0 = latc[ir],lonc[ir]
        lat1,lon1,lat2,lon2 = Po.CAMridgelet(lat=lat0 ,lon=lon0 , angll=angl[rank,ir], clngt=clng[rank,ir] )
        ax.plot( [lon1,lon2] , [lat1,lat2], color='black' ,transform=DataProj)

#ax.set_extent( [150. , 210., -85, -60] )
ax.set_extent( [ lonrange[0] ,lonrange[1] ,latrange[0] ,latrange[1]] )

gl = ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False  # Disable top labels
gl.right_labels = False  # Disable right labels
gl.xlabel_style = {'size': 12, 'color': 'gray'}
gl.ylabel_style = {'size': 12, 'color': 'gray'}


plt.title( f"Raw 3000x3000 topography over-plotted with 'ridgelets' from \n {CAM_Topo_FileN}")

#ax1 = fig.add_subplot(3, 4, ipoo , projection=ccrs.PlateCarree(central_longitude=0))
#cf = ax1.contourf(xlon, xlat, gutnx[L,:,:],  levels=clevs, cmap=cmap  , transform=ccrs.PlateCarree())

In [ ]:
def find_closest_point(lat_array, lon_array, lat1, lon1):
    # Calculate the squared distance between the target point and all points in the array
    dist_squared = (lat_array - lat1)**2 + (lon_array - lon1)**2
    
    # Find the indices of the minimum distance
    min_index = np.unravel_index(np.argmin(dist_squared), lat_array.shape)
    
    return min_index


In [ ]:
swco = find_closest_point(tlats[ip,:,:], tlons[ip,:,:], latrange[0], lonrange[0] )
neco = find_closest_point(tlats[ip,:,:], tlons[ip,:,:], latrange[1], lonrange[1] )
seco = find_closest_point(tlats[ip,:,:], tlons[ip,:,:], latrange[0], lonrange[1] )
nwco = find_closest_point(tlats[ip,:,:], tlons[ip,:,:], latrange[1], lonrange[0] )

print(tlons[ip,swco[0],swco[1]],tlats[ip,swco[0],swco[1]])
print(tlons[ip,neco[0],neco[1]],tlats[ip,neco[0],neco[1]])
print(neco)

#xxrange = [ np.min( swco[1], neco[1] ) , np.max( swco[1], neco[1] ) ]
xxrange = [ np.min( [ swco[1], neco[1], seco[1], nwco[1] ] ) , np.max( [ swco[1], neco[1] , seco[1], nwco[1] ] ) ]
yyrange = [ np.min( [ swco[0], neco[0], seco[0], nwco[0] ] ) , np.max( [ swco[0], neco[0] , seco[0], nwco[0] ] ) ]
print(xxrange)
print(yyrange)



In [ ]:
#plt.figure( figsize=(21,21))
plt.figure( figsize=(15,15))

#plt.contour( tlons[ip,:,:],tlats[ip,:,:],dev[ip,:,:], levels=np.linspace(-100,500.,num=21), cmap='terrain' ,alpha=.5) #,
plt.contourf( tlons[ip,:,:],tlats[ip,:,:],raw[ip,:,:], levels=np.linspace(-100,4000.,num=21), cmap='terrain',alpha=.5) #,
#plt.xlim(lonrange)
#plt.ylim(latrange)
#plt.plot( [40,90],[0,30])
for ir in iroo:
    lat0,lon0 = latc[ir],lonc[ir]
    lat1,lon1,lat2,lon2 = Po.CAMridgelet(lat=lat0 ,lon=lon0 , angll=angl[0,ir], clngt=clng[0,ir] )
    plt.plot( [lon1,lon2] , [lat1,lat2], color='black' )

plt.title( f"Raw 3000x3000 topography over-plotted with 'ridgelets' from \n {CAM_Topo_FileN}")

#ax1 = fig.add_subplot(3, 4, ipoo , projection=ccrs.PlateCarree(central_longitude=0))
#cf = ax1.contourf(xlon, xlat, gutnx[L,:,:],  levels=clevs, cmap=cmap  , transform=ccrs.PlateCarree())

In [ ]:
plt.contour( raw[ip,yyrange[0]:yyrange[1],xxrange[0]:xxrange[1]] ) #500:1500,1000:2000] )#1000:2000] )

In [ ]:
Po.sixpanel(aa=raw, clev=np.linspace(0,5000.,num=21) ,goofy=True , cmap='terrain' )
#Po.sixpanel(aa=yoog, clev=np.linspace(-1,2,num=21) )

## Read in 'remap' file. Currently an unformatted fortran file

In [ ]:
# RemapFile = 'remap_nc0540_Nsw008_Nrs000_Co012_Fi000_vX_20230106_1205.dat'
# FileN=TopoRoot +'/cases/'+ Case + '/output/'+RemapFile

# Do a file search in output directory
# Pick last (most recent) file
foo=glob.glob( TopoRoot +'/cases/'+ Case + '/output/remap*dat')
FileN = foo[-1]

f=FortranFile( FileN, 'r')
idims=f.read_record( '<i4' )

nc=idims[0]

uniqi=f.read_record( '<f8'   ).reshape( 6,nc,nc)
aniso=f.read_record( '<f8'   ).reshape( 6,nc,nc)
anglx=f.read_record( '<f8'   ).reshape( 6,nc,nc)
mxdis=f.read_record( '<f8'   ).reshape( 6,nc,nc)
hwdth=f.read_record( '<f8'   ).reshape( 6,nc,nc)
clngt=f.read_record( '<f8'   ).reshape( 6,nc,nc)
block=f.read_record( '<f8'   ).reshape( 6,nc,nc)
profi=f.read_record( '<f8'   ).reshape( 6,nc,nc)
nodes=f.read_record( '<f8'   ).reshape( 6,nc,nc)
wedge=f.read_record( '<f8'   ).reshape( 6,nc,nc)
nodos=f.read_record( '<f8'   ).reshape( 6,nc,nc)
wedgo=f.read_record( '<f8'   ).reshape( 6,nc,nc)



In [ ]:
print(np.shape(foo))
print(foo[-1])

In [ ]:
#plt.figure(figsize=(20, 20))
fig,ax = plt.subplots(1,2,figsize=(30,15))
ax[0].contourf( block[3,:,:])
#ax[0].contour( tlons[4,:,:], colors='white',clevels=51)
ax[0].set_aspect('equal') #,'box')
#ax[1].contourf( block[4,:,:])
ax[1].contourf( wedge[3,:,:])
ax[1].set_aspect('equal') #,'box')

In [ ]:
#plt.plot( dev[3,2800,:])
residual = dev - nodos #wedge


In [ ]:
#plt.figure(figsize=(20, 20))
clevels=np.linspace(-500.,3500.,num=21)
cmap=plt.cm.terrain

ipan=4
xlims=(0,3000) #( 200,1200 )
ylims=(0,3000) #( 2000, 3000 )


fig,ax = plt.subplots(2,2,figsize=(30,30) )
ax=ax.flatten()

ax[0].contourf( dev[ipan,:,:]  ,levels=clevels,cmap=cmap )
ax[0].set_xlim( xlims )
ax[0].set_ylim( ylims )
ax[0].set_aspect('equal') #,'box')

ax[1].contourf( profi[ipan,:,:] ,levels=clevels,cmap=cmap)
ax[1].set_xlim(  xlims )
ax[1].set_ylim(  ylims )
ax[1].set_aspect('equal') #,'box')

ax[2].contourf( mxdis[ipan,:,:] ,levels=clevels,cmap=cmap)
ax[2].set_xlim( xlims )
ax[2].set_ylim( ylims )
ax[2].set_aspect('equal') #,'box')

ax[3].contourf( (dev-profi)[ipan,:,:] ,levels=clevels,cmap=cmap)
ax[3].set_xlim( xlims )
ax[3].set_ylim( ylims )
ax[3].set_aspect('equal') #,'box')



In [ ]:
ipanel=3

xlim=(500,1000) #1000
ylim=(2500,3000)
#xlim=(0,3000)
#ylim=(0,3000)

clevels=np.linspace(-500.,3500.,num=21)
cmap=plt.cm.terrain
fig,ax = plt.subplots(2,2,figsize=(15,15) ,squeeze=False ) # , sharex=True)

titlesize=16


i,j=0,1
cf=ax[i,j].contourf( dev[ipanel,:,:],cmap=cmap,levels=clevels)
#cf=ax[0,0].contour( mxdis[ipanel,:,:],colors='black',levels=clevels)
#cf=ax[0,0].contourf( raw[ipanel,:,:],cmap=cmap,levels=clevels) #,alpha=.99)
#ax[0].contour( tlons[4,:,:], colors='white',clevels=51)
#plt.colorbar(cf,ax=ax[0,0],location='bottom')
ax[i,j].set_aspect('equal') #,'box')
ax[i,j].set_xlim( xlim )
ax[i,j].set_ylim( ylim )
ax[i,j].set_title( "Topographic deviation from smoothed Topo for bottom BC" , fontsize=titlesize )

#ax[1].contourf( block[4,:,:])
#ax[1].set_aspect('equal') #,'box')


i,j=1,1
"""
cf=ax[i,j].contourf( wedgo[ipanel,:,:],cmap=cmap,levels=clevels)
ax[i,j].set_aspect('equal') #,'box')
ax[i,j].set_xlim( xlim )
ax[i,j].set_ylim( ylim )
ax[i,j].set_title( "Volume filling reconstruction from ridge parameters" , fontsize=titlesize )
"""
cf=ax[i,j].contourf( mxdis[ipanel,:,:],cmap=cmap,levels=clevels)
ax[i,j].set_aspect('equal') #,'box')
#ax[i,j].set_xlim( (2200,2700) )
#ax[i,j].set_ylim( (500,1000) )
ax[i,j].set_ylim( (0,1000) )
ax[i,j].set_xlim( left=0,right=3000 , auto=True)
ax[i,j].set_title( "Ridge-line skeleton" , fontsize=titlesize )


i,j=0,0
cf=ax[i,j].contourf( raw[ipanel,:,:],cmap=cmap,levels=clevels)
ax[i,j].set_aspect('equal') #,'box')
ax[i,j].set_xlim( xlim )
ax[i,j].set_ylim( ylim )
ax[i,j].set_title( "Raw 3km X 3km Topography" , fontsize=titlesize )

i,j=1,0
cf=ax[i,j].contourf( mxdis[ipanel,:,:],cmap=cmap,levels=clevels)
ax[i,j].set_xlim( xlim )
ax[i,j].set_ylim( ylim )
ax[i,j].set_aspect('equal') #,'box')
ax[i,j].set_title( "Ridge-line skeleton" , fontsize=titlesize )

i,j=1,1
ipanel=0
cf=ax[i,j].contourf( mxdis[ipanel,:,:],cmap=cmap,levels=clevels)
#ax[i,j].set_xlim( (2200,2700) )
#ax[i,j].set_ylim( (500,1000) )
ax[i,j].set_xlim( (0,3000) )
ax[i,j].set_ylim( (0,3000) )
ax[i,j].set_aspect('equal') #,'box')
#ax[i,j].set_ylim( (0,1000) )
#ax[i,j].set_xlim( left=0,right=3000 , auto=True)
ax[i,j].set_title( "Ridge-line skeleton" , fontsize=titlesize )


plt.tight_layout()

plt.colorbar(cf,ax=ax,location='bottom',shrink=.4)



## Top-left shows raw 3km X 3km topography. For topographic BC generation, a smoothing scale is chosen first.  For 1 degree grids topography is typically smoothed over a ~180km radius. Top-right shows "deviation" of raw topography from topography smoothed with a 180km radius. Bottom-left shows "skeleton" derived by analyzing varaiances in rotated boxes with 180/sqrt(2) sides. Ridge-orientation is defined as the rotation angle with maximum variance of ridge profile (Y-average of topography, where Y is the coordinate along the potential ridge line).  Bottom-left shows a volume-filling reconstruction of the topography based on ridge parameters - heght, orientation, width, length

In [ ]:
ipanel=3

xlim=(300,1900)
ylim=(1400,3000)

clevels=np.linspace(-500.,3500.,num=21)
cmap=plt.cm.terrain
fig,ax = plt.subplots(1,1,figsize=(15,15) ,squeeze=False , sharex=True)

titlesize=16


i,j=0,0
cf=ax[i,j].contour( raw[ipanel,:,:],colors='white',levels=[1,10000])
cf=ax[i,j].contour( mxdis[ipanel,:,:],colors='black',levels=clevels)
cf=ax[i,j].contourf( dev[ipanel,:,:],cmap=cmap,levels=clevels)
#cf=ax[0,0].contourf( raw[ipanel,:,:],cmap=cmap,levels=clevels) #,alpha=.99)
#ax[0].contour( tlons[4,:,:], colors='white',clevels=51)
#plt.colorbar(cf,ax=ax[0,0],location='bottom')
ax[i,j].set_aspect('equal') #,'box')
ax[i,j].set_xlim( xlim )
ax[i,j].set_ylim( ylim )
ax[i,j].set_title( "Topographic deviation with skeleton over-plotted" , fontsize=titlesize )

ax[i,j].annotate("Nevada",
            xy=(700, 2900), xycoords='data',
            xytext=(1200, 2700), textcoords='data',color='white',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3",color='white'),
            )
#ax[1].contourf( block[4,:,:])
#ax[1].set_aspect('equal') #,'box')
plt.colorbar(cf,ax=ax,location='right',shrink=.4)

## Ridge skeleton overlaid on topographic deviation. Notice that deviation is high-pass and multi-scale, in other words, all scales smaller than the smoothing scale are retained. But, ridge algorithm tends to pick out features with scales on the order of the smoothng radius.The algorithm can pick out smaller scales only where no feature with scale~smoothing radius is present, e.g., Nevada

In [ ]:
import numpy as np

def coarse_grain(array, block_size):
    # Get the shape of the input array
    m, n = array.shape
    
    # Ensure the input array can be evenly divided into blocks
    assert m % block_size == 0 and n % block_size == 0, "Array dimensions must be divisible by the block size"
    
    # Reshape the array to a 4D array (blocks)
    reshaped_array = array.reshape(m // block_size, block_size, n // block_size, block_size)
    
    # Compute the mean of each block
    coarse_grained_array = reshaped_array.mean(axis=(1, 3))
    
    return coarse_grained_array

# Example usage
large_array = np.random.rand(3000, 3000)  # Replace with your actual array
block_size = 30
coarse_grained_array = coarse_grain(large_array, block_size)

print(coarse_grained_array.shape)  # Should output (100, 100)
